In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [3]:
string_vars = ["DISPOSITION_DESC"]

groupby_cols = ["CUSTOMER_ID", "FECHA"]

id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

past_periods = [7, 14, 21, 28, 84, 168]

In [11]:
def create_master_echi(echi: SQLPartitionedDataSet,
                       cliente_activo: pd.DataFrame,
                       parameters: Dict,
                       date: str) -> pd.DataFrame:
    """Creates master table with features related to interactions with call center for one period of data
    Parameters
    ----------
    cliente_activo: 
        dataset defined in ``catalog_raw.yml`` - list of active customers at EoP for the given period
    echi: 
        dataset defined in ``catalog_raw.yml`` with raw data information related to interactions with call center
    date: 
        period to process
    parameters: 
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        Master table with call center interactions features for one period
    """

    # Initialize logger
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "echi"

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_echi_dummies_past = pd.read_parquet(match[0], engine="pyarrow")

    else:
        # Read parameters
        lookback_days = parameters["masters"]["global"]["look_back_days"]
        start_date = pd.to_datetime(date) - timedelta(days=lookback_days)

        # Calculate period to load for active clients
        log.info("Loading table clientes")
        period_to_load = get_previous_month(date)
        df_clientes = cliente_activo[["CUSTOMER_ID"]]

        log.info("Loading table echi")
        df_echi = echi.filter_by(date=[start_date.strftime("%Y%m%d"),
                                       date])

        log.info("Merging with EOP...")
        df_echi = pd.merge(df_clientes,
                           df_echi,
                           on="CUSTOMER_ID",
                           validate="1:m")

        # Formatting variables
        df_echi["FECHA"] = df_echi["TMP_FECHA"].dt.strftime("%Y%m%d")
        log.info("Formatting string vars...")
        for var in string_vars:
            format_string_variable(df_echi, var)

        # Calculate daily stats
        df_echi_daily = flatten_df(df_echi.groupby(groupby_cols).agg({"SKILL_DESDE": "nunique",
                                                                      "DURATION": ["mean", "max"],
                                                                      "TALKTIME": ["mean", "max"],
                                                                      "HOLDABN": "sum"}))

        df_echi_daily.rename(columns={"SKILL_DESDE_nunique": "N_INSTANCIAS",
                                      "HOLDABN_sum": "N_HOLDS"}, inplace=True)

        # Create ratio between talktime and call duration
        create_evolution_variables(df_echi_daily,
                                   var_name="TALK_DUR_RATIO",
                                   numerator="TALKTIME_mean",
                                   denominator="DURATION_mean"
                                   )
        impute_categories(df_echi,
                          "DISPOSITION_DESC",
                          ["ABANDONED", "ANSWERED", "CONNECTED",
                           "FORCED_DISCONNECT", "INTERFLOWED"]
                          )

        log.info("Creating dummies...")
        log.info("---- DISPOSITION_DESC")
        df_echi_dummies = create_dummy_variables(df_echi,
                                                 var_to_dummy="DISPOSITION_DESC",
                                                 vars_to_groupby=groupby_cols,
                                                 include_total=False,
                                                 include_subtotal=False)

        # Merge
        df_echi_dummies = pd.merge(df_echi_daily,
                                   df_echi_dummies,
                                   on=groupby_cols,
                                   how="inner",
                                   validate="1:1")

        log.info("Calculating past variables...")
        # Calculate past variables
        df_echi_dummies_past = add_relative_calculate_past(df_echi_dummies,
                                                   date_col="FECHA",
                                                   id_cols=["CUSTOMER_ID"],
                                                   start_date=start_date,
                                                   end_date=date,
                                                   periods=past_periods,
                                                   agg={"N_INSTANCIAS": np.nansum,
                                                        "N_HOLDS": np.nansum,
                                                        "DURATION_mean": np.nanmean,
                                                        "DURATION_max": np.nanmax,
                                                        "TALK_DUR_RATIO": [np.nanmean, np.nanmax],
                                                        "DISPOSITION_DESC_ABANDONED": np.nansum,
                                                        "DISPOSITION_DESC_ANSWERED": np.nansum,
                                                        #"DISPOSITION_DESC_CONNECTED": np.nansum, En uruguay no se cuenta con este campo
                                                        "DISPOSITION_DESC_FORCED_DISCONNECT": np.nansum,
                                                        "DISPOSITION_DESC_INTERFLOWED": np.nansum
                                                        },
                                                   period_freq="D")

        # Add date variables
        df_echi_dummies_past["DATE_EXP"] = period_to_load
        df_echi_dummies_past["DATE_CALC"] = date

        # Change variable names 
        table_preffix = parameters["masters"]["echi"]["table_preffix"]
        rename_table(df_echi_dummies_past,
                     preffix=table_preffix,
                     ids_to_exclude=id_cols)
        log.info(f"Exporting {df_echi_dummies_past.shape[0]} rows and {df_echi_dummies_past.shape[1]} columns")

        if write_to_parquet:
            log.info(f"\n------ Writing {table_name} for period {date} to parquet ------")
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_echi_dummies_past.to_parquet(file, engine="pyarrow")

        log.info(f"Exporting {df_echi_dummies_past.shape[0]} rows and {df_echi_dummies_past.shape[1]} columns")

    return df_echi_dummies_past

In [5]:
cliente_activo=catalog.load("cliente_activo")
echi= catalog.load("echi")

2020-12-30 15:22:45,113 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...
2020-12-30 15:22:45,115 - kedro.io.data_catalog - INFO - Loading data from `echi` (SQLPartitionedDataSet)...


In [6]:
dates = calculate_dates_to_process_for_master(parameters, table_name="echi")

In [13]:
for date in dates:
    print(f"Processing date {date}")
    cliente_activo_df= create_cliente_activo(cliente_activo,date)
    create_master_echi(echi,cliente_activo_df,parameters,date)
    

Processing date 20181203
2020-12-30 15:25:30,669 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201811
2020-12-30 15:25:34,828 - aa_engine_pkg.assets.utils.utilities - INFO - Loading table clientes
2020-12-30 15:25:34,831 - aa_engine_pkg.assets.utils.utilities - INFO - Loading table echi
select CLI_IBS_ID as CUSTOMER_ID, TMP_FECHA, DISPOSITION_DESC, SKILL_DESDE, DURATION, TALKTIME, HOLDABN from stg_uy_echi where TMP_FECHA >= to_date('20180618', 'yyyymmdd') and TMP_FECHA < to_date('20181203', 'yyyymmdd')
2020-12-30 15:25:53,187 - aa_engine_pkg.assets.utils.utilities - INFO - Merging with EOP...
2020-12-30 15:25:55,929 - aa_engine_pkg.assets.utils.utilities - INFO - Formatting string vars...
2020-12-30 15:25:58,069 - numexpr.utils - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-12-30 15:25:58,